# Description


# Modules and Global Variables

In [1]:
from transformers import (
    ElectraTokenizerFast, ElectraForSequenceClassification, 
    DataCollatorWithPadding, 
    TrainingArguments, Trainer,
)

import torch
import wandb

import datasets
import evaluate

import numpy as np
import pandas as pd

import os
import re
import random

import demoji

/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f'torch.__version__: {torch.__version__}')
print(f'torch.cuda.is_available(): {torch.cuda.is_available()}')
NGPU = torch.cuda.device_count()
print(f'NGPU: {NGPU}')

torch.__version__: 1.12.1
torch.cuda.is_available(): True
NGPU: 4


In [3]:
### labels

ce_labels = ['True', 'False']
pc_labels = ['positive', 'negative', 'neutral']
pc_binary_labels = ['True', 'False']

labels = ce_labels

label2id = {k: i for i, k in enumerate(labels)}
id2label = {i: k for i, k in enumerate(labels)}
num_labels = len(labels)

print(label2id)
print(id2label)

{'True': 0, 'False': 1}
{0: 'True', 1: 'False'}


In [4]:
### paths and names

PROJECT_NAME = 'aspect_category_detection'
RUN_ID = 'uncleaned_v13'

DATA_V = 'uncleaned_v13'
DATA_T = 'ce' # ce or pc or pc_binary
AUGMENTATION = False
AUG_NAME = 'balanced'

model_checkpoint = 'monologg/koelectra-base-v3-discriminator'

notebook_name = 'trainer_for_acd_binary.ipynb'

### fixed

model_name = re.sub(r'[/-]', r'_', model_checkpoint).lower()
run_name = f'{model_name}_{RUN_ID}'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, 'training_results', run_name, 'acd')
NOTEBOOK_PATH = os.path.join('./', notebook_name)

augornot = f'_{AUG_NAME}' if AUGMENTATION is True else ''
TRAIN_DATA_PATH = os.path.join(ROOT_PATH, 'dataset', DATA_V, f'{DATA_T}_train{augornot}.csv')
EVAL_DATA_PATH = os.path.join(ROOT_PATH, 'dataset', DATA_V, f'{DATA_T}_dev.csv')

!mkdir -p {SAVE_PATH}

In [5]:
if os.path.exists(SAVE_PATH):
    print(f'{SAVE_PATH} exists.')
else:
    print(f'{SAVE_PATH} does not exist.')
if os.path.exists(NOTEBOOK_PATH):
    print(f'{NOTEBOOK_PATH} exists.')
else:
    print(f'{NOTEBOOK_PATH} does not exist.')
if os.path.exists(TRAIN_DATA_PATH):
    print(f'{TRAIN_DATA_PATH} exists.')
else:
    print(f'{TRAIN_DATA_PATH} does not exist.')
if os.path.exists(EVAL_DATA_PATH):
    print(f'{EVAL_DATA_PATH} exists.')
else:
    print(f'{EVAL_DATA_PATH} does not exist.')

./training_results/monologg_koelectra_base_v3_discriminator_uncleaned_v13/acd exists.
./trainer_for_acd_binary.ipynb exists.
./dataset/uncleaned_v13/ce_train.csv exists.
./dataset/uncleaned_v13/ce_dev.csv exists.


In [6]:
### rest of training args

report_to="wandb"

fp16 = False

num_train_epochs = 20
batch_size = 32
gradient_accumulation_steps = 1

optim = 'adamw_hf' # 'adamw_torch'

learning_rate = 3e-6 / 8 * batch_size * NGPU # 5e-5
weight_decay = 0.01 # 0
adam_epsilon = 1e-8

lr_scheduler_type = 'linear'
warmup_ratio = 0

save_total_limit = 2

load_best_model_at_end = True
metric_for_best_model ='f1_macro'

save_strategy = "epoch"
evaluation_strategy = "epoch"

logging_strategy = "steps"
logging_first_step = True 
logging_steps = 100

print(learning_rate)

4.8e-05


# WandB Configuration

In [7]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

env: WANDB_PROJECT=aspect_category_detection
env: WANDB_NOTEBOOK_NAME=./trainer_for_acd_binary.ipynb
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin


True

# Load Model, Tokenizer, and Collator

In [8]:
tokenizer = ElectraTokenizerFast.from_pretrained(model_checkpoint)
model = ElectraForSequenceClassification.from_pretrained(
    model_checkpoint, label2id=label2id, id2label=id2label, num_labels=num_labels
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
train_path = f'./dataset/{DATA_V}/raw_train.csv'
dev_path = f'./dataset/{DATA_V}/raw_dev.csv'
test_path = f'./dataset/{DATA_V}/raw_test.csv'
train = pd.read_csv(train_path)
dev = pd.read_csv(dev_path)
test = pd.read_csv(test_path)

print(len(tokenizer))
tokenizer_train_data = pd.concat([train.sentence_form, dev.sentence_form, test.sentence_form]).to_frame().drop_duplicates()
tokenizer_train_data = tokenizer_train_data.sentence_form.to_list()
new_tokenizer = ElectraTokenizerFast.from_pretrained(model_checkpoint)
new_tokenizer = tokenizer.train_new_from_iterator(tokenizer_train_data, vocab_size=1)
new_tokens = set(list(new_tokenizer.vocab.keys())) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))
print(len(new_tokenizer))
print(len(tokenizer))
model.resize_token_embeddings(len(tokenizer))

35000





3060
35212


Embedding(35212, 768)

In [10]:
print(len(new_tokens))
print(new_tokens)

212
{'##🌝', '##🖒', '##읒', '😡', '죱', 'ᴡ', '##🧚', '♪', '👌', '꺠', '쨕', '##쫜', 'ᴏ', '옄', '💬', '##ᴇ', '🐱', '##ɴ', '##🚗', '귯', '쏨', '💝', '😺', 'ɢ', '##띡', 'ᴘ', '곘', '🙃', '닠', '˃', '쫜', '🍼', '💇', '##®', '🌝', '💞', '##ᴡ', 'ෆ', '🚗', '##잍', '##💝', '뿤', '🔸', '##펏', '##👨', '##ᴘ', '챦', '웻', '##ᴏ', '##ˇ', '##젔', '##➰', '㉦', '🤡', '##쵝', '🎀', '##👠', 'ʀ', '🧚', '🖐', '🍎', '##곘', '뜌', 'ㅖ', 'ˇ', '⏰', '##죱', '➕', '##ɢ', '횽', '챳', '##👆', '🕺', 'ꈍ', '##ꈍ', '##💆', '🐄', '🕸', '🤟', '💧', '🥤', '##🕸', '◍', '##🍎', '띡', '똭', '##➕', '##💇', '👨', '##듕', '##🤮', '🕷', '##옄', '##☝', '##닠', 'ɪ', '##ʀ', '̵', '🎂', '👠', '##🤘', 'ʜ', '퐉', '##˚', '##😲', '💄', '##🤡', '##🎵', '♩', '##앝', '##💄', '##◍', '👦', '♬', '˂', 'ᴠ', '📸', '##💞', '##˂', '##💋', '◡', 'ღ', '##̵', '💡', '읒', '##😶', '👆', '##ᴍ', '##촥', '☝', '##ᴜ', '💯', 'ᴗ', '듕', '##❔', '##ᴀ', '##♪', '🤘', '##◡', '잍', '젔', '##🎂', '핡', '🌹', '##츌', '##🌹', '##귯', '😲', '𖤐', 'ᵕ', '˚', '😮', '##ɪ', '😴', '졓', '⁉', '💋', '##♩', '##💧', '##👌', '##쨕', '##㉦', '❔', '🐥', '밪', '##♬', '##😬', '##ᴛ', '͈', 'ᴇ', 'ᴛ'

In [11]:
model.config.label2id, model.config.id2label, model.num_labels

({'True': 0, 'False': 1}, {0: 'True', 1: 'False'}, 2)

# Define Metric

In [12]:
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_metric.compute(references=labels, predictions=predictions)['accuracy']
    f1_true, f1_false = tuple(f1_metric.compute(references=labels, predictions=predictions, average=None, labels=[0,1])['f1'])
    f1_macro = f1_metric.compute(references=labels, predictions=predictions, average='macro')['f1']
    f1_micro = f1_metric.compute(references=labels, predictions=predictions, average='micro')['f1']
    
    return {'accuracy': accuracy, 'f1_true': f1_true, 'f1_false': f1_false, 'f1_macro': f1_macro, 'f1_micro': f1_micro}

# Load Data

In [14]:
def preprocess_function(examples):
    return tokenizer(examples["form"], examples["pair"], truncation=True)

In [15]:
train_dataset = pd.read_csv(TRAIN_DATA_PATH)
eval_dataset = pd.read_csv(EVAL_DATA_PATH)
train_dataset = datasets.Dataset.from_pandas(train_dataset) #.shuffle(seed=42)
eval_dataset = datasets.Dataset.from_pandas(eval_dataset) #.shuffle(seed=42)
train_dataset = train_dataset.map(preprocess_function, batched=False)
eval_dataset = eval_dataset.map(preprocess_function, batched=False)

  0%|          | 0/75000 [00:00<?, ?ex/s]

  1%|          | 472/75000 [00:00<00:15, 4710.06ex/s]

  1%|▏         | 944/75000 [00:00<00:15, 4669.01ex/s]

  2%|▏         | 1411/75000 [00:00<00:17, 4187.38ex/s]

  2%|▏         | 1874/75000 [00:00<00:16, 4347.98ex/s]

...

 98%|█████████▊| 68202/69825 [00:16<00:00, 4131.48ex/s]

 98%|█████████▊| 68721/69825 [00:16<00:00, 4426.81ex/s]

 99%|█████████▉| 69169/69825 [00:16<00:00, 4379.01ex/s]

100%|█████████▉| 69610/69825 [00:16<00:00, 4374.37ex/s]

100%|██████████| 69825/69825 [00:16<00:00, 4226.93ex/s]

In [16]:
len(train_dataset), len(eval_dataset)

(75000, 69825)

In [17]:
k = random.randrange(len(train_dataset))
print(tokenizer.decode(train_dataset['input_ids'][k]), train_dataset['labels'][k])
k = random.randrange(len(eval_dataset))
print(tokenizer.decode(eval_dataset['input_ids'][k]), eval_dataset['labels'][k])

[CLS] 그리고 인스타에서도 완전 인기몰이중 ~ ~ ~ [SEP] 본품 # 디자인 [SEP] 1


[CLS] 쫀쫀함이 느껴지는 점성있는 제형이라 피부에 닿았을 때 너무 부드럽고 흡수 된 후에는 끈적이지 않고 산뜻하네용 ㅎㅎ [SEP] 본품 # 품질 [SEP] 0


# Load Trainer

In [18]:
args = TrainingArguments(
    output_dir=run_name,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,
    
    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,
    
    fp16=fp16,
)

In [19]:
# es = EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    # callbacks=[es],
)

# Run Trainer

In [21]:
trainer.train()
wandb.finish()

The following columns in the training set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: id, form, pair. If id, form, pair are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.


/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****


  Num examples = 75000


  Num Epochs = 20


  Instantaneous batch size per device = 32


...

Configuration saved in /tmp/tmpn4q2ccvb/config.json


Model weights saved in /tmp/tmpn4q2ccvb/pytorch_model.bin


tokenizer config file saved in /tmp/tmpn4q2ccvb/tokenizer_config.json


Special tokens file saved in /tmp/tmpn4q2ccvb/special_tokens_map.json


eval/accuracy,▁▆▅▅▇▆▅▇▃▄▅▅▆▆▇█▇███
eval/f1_false,▁▆▅▅▇▆▅▇▃▄▄▅▆▆▇█▇███
eval/f1_macro,▁▂▅▆▆█▇▆▄▆▆▇█▇█▇████
eval/f1_micro,▁▆▅▅▇▆▅▇▃▄▅▅▆▆▇█▇███
eval/f1_true,▁▂▅▆▆█▇▆▄▆▆▇█▇█▇████
eval/loss,▁▁▂▂▂▃▄▄▆▆▅▆▆█▇██▇██
eval/runtime,█▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▂▁▁
eval/samples_per_second,▁█▇█████▇▇███████▇█▇
eval/steps_per_second,▁█▇█████▇▇███████▇█▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [22]:
keep = [
    'added_tokens.json',
    'config.json',
    'pytorch_model.bin',
    'special_tokens_map.json',
    'tokenizer.json',
    'tokenizer_config.json',
    'vocab.txt'
]

ckpts = os.listdir(run_name)
for ckpt in ckpts:
    ckpt = os.path.join(run_name, ckpt)
    for item in os.listdir(ckpt):
        if item not in keep:
            os.remove(os.path.join(ckpt, item))

!mv wandb {run_name} {SAVE_PATH}/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
